Importações e configuração

In [18]:
import pandas as pd
from pathlib import Path
# NOVAS IMPORTAÇÕES NECESSÁRIAS PARA MINIO/S3
from minio import Minio
from minio.error import S3Error
import io # Para manipular streams de bytes lidos do MinIO

# --- CONFIGURAÇÃO DE AMBIENTE E CAMINHOS ---

# Caminho de saída (continua sendo local)
BASE_PROC = Path("/home/jovyan/data/processed")
BASE_PROC.mkdir(parents=True, exist_ok=True)


# --- CONFIGURAÇÃO DO MINIO (Data Lake) ---
# O endpoint 'minio:9000' é o nome do serviço na rede Docker Compose
MINIO_ENDPOINT = "minio:9000" 
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "admin12345"
RAW_BUCKET = "inmet-raw"
# Prefixo que você usou no script de upload
RAW_PREFIX = "raw/" 

# Cria o cliente MinIO (executado dentro do contêiner Jupyter)
try:
    minio_client = Minio(
        MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False,
    )
    print("✅ Conexão MinIO estabelecida.")
    if not minio_client.bucket_exists(RAW_BUCKET):
        print(f"❌ Erro: O bucket '{RAW_BUCKET}' não existe. Verifique o serviço MinIO.")
except Exception as e:
    print(f"❌ Erro ao conectar ao MinIO: {e}")

✅ Conexão MinIO estabelecida.


Processamento

In [19]:
def processar_inmet(file_content: bytes, file_name: str) -> pd.DataFrame:
    """
    Processa o conteúdo de um arquivo CSV bruto do INMET, lendo-o de um stream de bytes (MinIO/S3).
    Aplica limpeza de dados, conversão de tipos, tratamento de faltantes por interpolação.
    """
    print(f"\n📄 Lendo arquivo: {file_name}")
    
    # 1) LER CSV DO STREAM DE BYTES
    df = pd.read_csv(
        io.BytesIO(file_content), # <--- MUDANÇA AQUI: Lê do stream de bytes
        sep=';',
        skiprows=8,           # pula bloco de metadados
        encoding='latin1',
        low_memory=False
    )
    
    # 2) Renomear colunas de interesse (só as que existirem)
    mapa_renome = {
        'Data': 'data',
        'Hora UTC': 'hora',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp_ar',
        'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
        'RADIACAO GLOBAL (Kj/m²)': 'radiacao',
        'RADIACAO GLOBAL (kJ/m²)': 'radiacao',
        'RADIACAO GLOBAL': 'radiacao',
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'vento_vel',
        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'vento_dir',
        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
    }
    df = df.rename(columns={orig: novo for orig, novo in mapa_renome.items() if orig in df.columns})

    # 3) Garante que data e hora existem
    if 'data' not in df.columns or 'hora' not in df.columns:
        raise ValueError("Colunas 'Data' e/ou 'Hora UTC' não foram encontradas após o rename.")

    # 4) Padronizar 'data' e 'hora' como string
    df['data'] = df['data'].astype(str).str.strip()
    df['hora'] = df['hora'].astype(str).str.strip()

    # 5) Remover ' UTC' da hora, se existir
    df['hora'] = df['hora'].str.replace(' UTC', '', regex=False)

    # 6) Se hora estiver como 0, 300, 1200 etc, padronizar para HH:MM
    mascara_numerica = df['hora'].str.fullmatch(r'\d{1,4}')
    df.loc[mascara_numerica, 'hora'] = (
        df.loc[mascara_numerica, 'hora']
          .str.zfill(4)
          .str[:2] + ':' + df.loc[mascara_numerica, 'hora'].str.zfill(4).str[2:]
    )

    # 7) Criar coluna datetime (deixa o pandas inferir formato)
    df['datetime'] = pd.to_datetime(
        df['data'] + ' ' + df['hora'],
        errors='coerce'
    )

    # Remover linhas onde datetime não pôde ser montado
    df = df.dropna(subset=['datetime'])
    df = df.set_index('datetime')

    # 8) Selecionar colunas de interesse (as que existirem)
    colunas_desejadas = ['hora', 'temp_ar', 'umidade', 'radiacao',
                         'vento_vel', 'precipitacao', 'pressao']
    colunas_presentes = [c for c in colunas_desejadas if c in df.columns]
    df = df[colunas_presentes]

    # 9) Converter variáveis numéricas para float
    variaveis_numericas = [c for c in colunas_presentes if c != 'hora']

    for col in variaveis_numericas:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 10) Remover colunas que estão 100% NaN 
    colunas_todas_nan = [c for c in variaveis_numericas if df[c].isna().sum() == len(df)]
    if colunas_todas_nan:
        print("   ⚠ Removendo colunas 100% NaN:", colunas_todas_nan)
        df = df.drop(columns=colunas_todas_nan)
        variaveis_numericas = [c for c in variaveis_numericas if c not in colunas_todas_nan]

    # 11) Interpolar valores faltantes ao longo do tempo
    if variaveis_numericas:
        df[variaveis_numericas] = df[variaveis_numericas].interpolate(
            method='time'
        )

        # Opcional: preencher pontas (início/fim) com ffill/bfill
        df[variaveis_numericas] = df[variaveis_numericas].ffill().bfill()

    # 12) Remover apenas linhas em que TODAS as variáveis numéricas são NaN (caso sobrem)
    if variaveis_numericas:
        mask_all_nan = df[variaveis_numericas].isna().all(axis=1)
        n_all_nan = mask_all_nan.sum()
        if n_all_nan > 0:
            print(f"   ⚠ Removendo {n_all_nan} linhas 100% NaN nas variáveis numéricas.")
            df = df[~mask_all_nan]

    # 13) Criar features auxiliares
    df['hora_num'] = df.index.hour
    df['mes'] = df.index.month

    print(f"   ✅ Registros após tratamento: {len(df)}")
    return df

Loop para processar Petrolina e Garanhuns (2020 a 2024)

In [20]:
# Definição dos anos e cidades para iteração
anos = ['2020', '2021', '2022', '2023', '2024']
cidades = ['PETROLINA', 'GARANHUNS']
print(f"Anos a processar: {anos}")
print(f"Cidades a processar: {cidades}")

for ano in anos:
    for cidade in cidades:
        estacao_id = cidade.lower()
        
        # 1. Lista Objetos no MinIO
        # Busca por prefixo: raw/<ano>/
        search_prefix = f"{RAW_PREFIX}{ano}/"
        
        objetos_minio = minio_client.list_objects(
            RAW_BUCKET, 
            prefix=search_prefix, 
            recursive=True
        )

        # Filtra os objetos que contêm o nome da cidade e terminam com .CSV
        arquivos_encontrados = [obj for obj in objetos_minio 
                                if cidade.upper() in obj.object_name.upper() and obj.object_name.endswith(".CSV")]

        if not arquivos_encontrados:
            print(f"\n🚫 Nenhum arquivo encontrado para {cidade} em {ano} no MinIO com prefixo {search_prefix}")
            continue

        for obj in arquivos_encontrados:
            response = None 
            try:
                # 2. Baixa o conteúdo do arquivo do MinIO (como bytes)
                print(f"\n📥 Baixando {obj.object_name}...")
                response = minio_client.get_object(RAW_BUCKET, obj.object_name)
                # Lê o conteúdo completo do stream
                file_content = response.read()
                
                # Pega apenas o nome do arquivo para usar no print de log
                file_name_short = obj.object_name.split('/')[-1]

                # 3. Processamento e Limpeza (Função modificada recebe bytes)
                df_tratado = processar_inmet(file_content, file_name_short)
                
                # --- PREPARAÇÃO PARA O SNOWFLAKE (Mantida do seu original) ---
                
                # 3a. Adiciona a coluna ESTACAO_ID
                df_tratado['ESTACAO_ID'] = estacao_id
                
                # 3b. Renomeia colunas para bater com o DDL final
                mapa_cols_snowflake = {
                    'pressao': 'PRESSAO',
                    'radiacao': 'RADIACAO',
                    'vento_vel': 'VENTO_VEL',
                    'precipitacao': 'PRECIPITACAO',
                    'temp_ar': 'TEMP_AR',
                    'umidade': 'UMIDADE',
                    'hora_num': 'HORA_NUM',
                    'mes': 'MES',
                    'hora': 'HORA'
                }

                df_final = df_tratado.reset_index().rename(columns=mapa_cols_snowflake)
            
                if 'index' in df_final.columns:
                    df_final = df_final.drop(columns=['index'])

                df_final.rename(columns={'datetime': 'DATETIME'}, inplace=True)
                
                # 3c. Garante a ordem das colunas
                colunas_ddl = [
                    'DATETIME', 'ESTACAO_ID', 'TEMP_AR', 'UMIDADE', 'PRESSAO', 
                    'VENTO_VEL', 'RADIACAO', 'PRECIPITACAO', 'HORA_NUM', 'MES', 'HORA'
                ]
                
                colunas_existentes = [c for c in colunas_ddl if c in df_final.columns]
                df_final = df_final[colunas_existentes]


                # 4. Salva CSV Localmente (como arquivo processado)
                nome_saida = f"{estacao_id}_{ano}_tratado.csv"
                caminho_saida = BASE_PROC / nome_saida
                # Salva o dataframe tratado, usando o índice datetime
                df_tratado.to_csv(caminho_saida, index=True, index_label='DATETIME') 
                print(f"   💾 Salvo em: {caminho_saida}")
                
                # 5. Carga para o Snowflake
                # carregar_para_snowflake(df_final, SNOWFLAKE_TABLE)
                print(f"   ➡️ DataFrame pronto para carga no Snowflake (Tabela: {SNOWFLAKE_TABLE}).")


            except S3Error as e:
                print(f"   ❌ Erro S3 ao processar {obj.object_name}: {e}")
            except Exception as e:
                print(f"   ❌ Erro ao processar {obj.object_name}: {e}")
            finally:
                # É crucial fechar a conexão do stream do MinIO após o uso
                if response:
                    response.close()
                    response.release_conn()

Anos a processar: ['2020', '2021', '2022', '2023', '2024']
Cidades a processar: ['PETROLINA', 'GARANHUNS']

📥 Baixando raw/2020/INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV...

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2020_tratado.csv
   ➡️ DataFrame pronto para carga no Snowflake (Tabela: DADOS_INMET_HORARIO).

📥 Baixando raw/2020/INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV...

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2020_tratado.csv
   ➡️ DataFrame pronto para carga no Snowflake (Tabela: DADOS_INMET_HORARIO).

📥 Baixando raw/2021/INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV...

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV
   ✅ Registros após tratamento: 8760
   💾 Salvo em: /home/jovyan/da

Resumo

In [21]:
processed_path = BASE_PROC

arquivos_tratados = list(processed_path.glob("*_tratado.csv"))

print("Foram encontrados", len(arquivos_tratados), "arquivos tratados.\n")

for arquivo in arquivos_tratados:
    df = pd.read_csv(arquivo, index_col=0, parse_dates=True)
    
    print("📄 Arquivo:", arquivo.name)
    print("   ➤ Linhas:", len(df))
    print("   ➤ NaNs por coluna:")
    print(df.isna().sum().to_string())
    print("-" * 60)


Foram encontrados 10 arquivos tratados.

📄 Arquivo: petrolina_2020_tratado.csv
   ➤ Linhas: 8784
   ➤ NaNs por coluna:
hora            0
temp_ar         0
umidade         0
radiacao        0
vento_vel       0
precipitacao    0
pressao         0
hora_num        0
mes             0
ESTACAO_ID      0
------------------------------------------------------------
📄 Arquivo: petrolina_2022_tratado.csv
   ➤ Linhas: 8760
   ➤ NaNs por coluna:
hora            0
temp_ar         0
umidade         0
radiacao        0
vento_vel       0
precipitacao    0
pressao         0
hora_num        0
mes             0
ESTACAO_ID      0
------------------------------------------------------------
📄 Arquivo: garanhuns_2024_tratado.csv
   ➤ Linhas: 8784
   ➤ NaNs por coluna:
hora          0
temp_ar       0
umidade       0
radiacao      0
vento_vel     0
pressao       0
hora_num      0
mes           0
ESTACAO_ID    0
------------------------------------------------------------
📄 Arquivo: garanhuns_2023_tratado.csv


Nota:

Para lidar com valores climáticos faltantes nos arquivos do INMET, utilizamos a técnica de interpolação temporal, que é um método recomendado internacionalmente para reconstrução de séries meteorológicas. Em termos simples, a interpolação preenche lacunas usando os valores anteriores e posteriores como referência. Por exemplo: se às 10h a temperatura é 22°C e às 12h é 24°C, a interpolação estima que às 11h ela seria aproximadamente 23°C. Isso evita descartar horas inteiras — o que reduziria drasticamente a qualidade do conjunto de dados — e mantém a continuidade natural do clima, que muda de forma gradual ao longo do tempo. Esse procedimento foi aplicado usando df.interpolate(method='time'), que utiliza o índice de datas do próprio dataframe para estimar valores de forma coerente com a evolução temporal. O uso de interpolação em dados climáticos é amplamente adotado por instituições como o NOAA – National Oceanic and Atmospheric Administration, que afirma que a interpolação linear é adequada para preenchimento de falhas curtas em séries meteorológicas contínuas (NOAA, Climate Data Interpolation Guidelines, 2018).

CONFIGURAÇÃO POSTGRESQL

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# ============================
# CONFIGURAÇÃO POSTGRESQL
# ============================
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'dados_clima',
    'user': 'inmet_user',
    'password': 'inmet123'
}

# Criar engine do SQLAlchemy
engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

print("✅ Configuração PostgreSQL definida.")

Configurações do Snowflake definidas.


FUNÇÃO PARA CARREGAR DADOS NO POSTGRESQL

In [ ]:
# ==============================================================================
# FUNÇÃO PARA CARREGAR DADOS NO POSTGRESQL
# ==============================================================================

def carregar_para_postgres(df: pd.DataFrame, nome_tabela: str):
    """
    Carrega um DataFrame Pandas para a tabela PostgreSQL especificada.
    """
    print(f"\n🐘 Salvando dados na tabela {nome_tabela} do PostgreSQL...")
    
    try:
        # Salvar no PostgreSQL
        df.to_sql(
            name=nome_tabela,
            con=engine,
            if_exists='append',      # 'replace' para substituir, 'append' para adicionar
            index=False,             # salva o índice datetime
            index_label='datetime'
        )
        
        print(f"   ✅ Carga concluída! {len(df)} linhas salvas.")
        
    except Exception as e:
        print(f"   ❌ ERRO ao carregar: {e}")
        print("   Primeiras linhas do DataFrame com erro:")
        print(df.head())

LOOP PARA PROCESSAR, SALVAR CSV LOCALMENTE E CARREGAR NO POSTGRESQL

In [ ]:
# ==============================================================================
# LOOP PARA PROCESSAR, SALVAR CSV LOCALMENTE E CARREGAR NO POSTGRESQL
# ==============================================================================

for ano in anos:
    pasta_ano = BASE_RAW / ano
    
    for cidade in cidades:
        estacao_id = cidade.lower()
        
        arquivos = list(pasta_ano.glob(f"{cidade}.CSV"))
        
        if not arquivos:
            print(f"\n🚫 Nenhum arquivo encontrado para {cidade} em {ano}")
            continue
        
        for arq in arquivos:
            # 1. Processamento e Limpeza
            df_tratado = processar_inmet(arq)
            
            # 2. Ajustes finais
            df_tratado['estacao_id'] = estacao_id
            
            # 3. Resetar o índice para que datetime vire coluna
            df_final = df_tratado.reset_index()
            
            # 4. Salva CSV Localmente (opcional)
            nome_saida = f"{estacao_id}_{ano}_tratado.csv"
            caminho_saida = BASE_PROC / nome_saida
            df_tratado.to_csv(caminho_saida)
            print(f"   💾 Salvo em: {caminho_saida}")
            
            # 5. Carga para o PostgreSQL
            carregar_para_postgres(df_final, 'dados_inmet_horario')

print("\n🎉 Processamento completo!")


⚠️ Tabela DADOS_INMET_HORARIO truncada com sucesso no Snowflake.

📥 Baixando raw/2020/INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV...

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2020_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).

📥 Baixando raw/2020/INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV...

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2020_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).

📥 Baixando raw/2021/INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV...

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV
   ✅ Registros após tratamento: 8760
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2021_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


...

   ✅ Carga concluída. Linhas carregadas: 8760
   ✅ Arquivos enviados: 1 chunk(s).

📥 Baixando raw/2024/INMET_NE_PE_A307_PETROLINA_01-01-2024_A_31-12-2024.CSV...

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2024_A_31-12-2024.CSV
   ⚠ Removendo colunas 100% NaN: ['radiacao']
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2024_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).

📥 Baixando raw/2024/INMET_NE_PE_A322_GARANHUNS_01-01-2024_A_31-12-2024.CSV...

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2024_A_31-12-2024.CSV
   ⚠ Removendo colunas 100% NaN: ['precipitacao']
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2024_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).
